In [2]:
from hello.hello3 import HelloApp
from hello import hello3
from hello import hello
from hello.pid.ag_pid import agpid
from hello.pid.ag_pid.agpid import SimplePIDRunner, PIDRunner, OffToAutoTest, ManToAutoTest, AutoToAutoTest

In [5]:
with open(agpid.__file__, 'r') as f:
    src = f.read()
ns = {}
exec(src, ns, ns)
SimplePIDRunner = ns['SimplePIDRunner']
PIDRunner = ns['PIDRunner']
OffToAutoTest = ns['OffToAutoTest']
ManToAutoTest = ns['ManToAutoTest']
AutoToAutoTest = ns['AutoToAutoTest']
ManToManTest = ns['ManToManTest']

In [6]:
def runagpid(r):
    try:
        _runagpid(r)
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r

def _runagpid(r):
    r.doall()
    return r

def the80Lalreadyworks_settings():
    pgains = (0.1,)
    itimes = (0.01, 0.02, 0.05)
    dtimes = (0,)
    sps = (8, 12, 20)
    return pgains, itimes, dtimes, sps

def settings2():
    sps = 8, 12, 20
    c1 = [(.3, .02, 0, sp) for sp in sps]
    c2 = [(.1, .01, 0, sp) for sp in sps]
    return (), (), (), (), c1+c2

def run_the80Lalreadyworks(ip='71.189.82.196:85'):
    settings = the80Lalreadyworks_settings()
    app = HelloApp(ip)
    r = SimplePIDRunner(*settings, app_or_ipv4=app)
    try:
        runagpid(r)
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r
    
def run_80Lothertests(ip='71.189.82.196:85'):
    p = 0.1
    i = 0.01
    d = 0
    app = hello3.open_hello(ip)
    tests = []
    for sp in (8, 10, 12, 14):
        t = OffToAutoTest(p, i, d, sp, app)
        tests.append(t)
        break
    for start_pv in (10, 12, 14):
        t1 = ManToAutoTest(p, i, d, start_pv-2, app, start_pv)
        t2 = AutoToAutoTest(p, i, d, start_pv-2, app, start_pv)
        t3 = ManToManTest(p, i, d, start_pv-2, app, start_pv)
        tests.extend((t1, t2, t3))
        break
    for start_pv in (8, 10, 12):
        break
        t1 = ManToAutoTest(p, i, d, start_pv+2, app, start_pv)
        t2 = AutoToAutoTest(p, i, d, start_pv+2, app, start_pv)
        t3 = ManToManTest(p, i, d, start_pv+2, app, start_pv)
        tests.extend((t1, t2, t3))
    r = PIDRunner(tests, wb_name='test.xlsx')
    return runagpid(r)
    

In [ ]:
r = run_80Lothertests()

1484097892.322021 INFO <runall>: Running test 1 of 4: P:0.100 I:0.010 D:0.0000 SP:8
1484097892.324033 INFO <run>: Beginning test run for OffToAuto SP:8.0
1484097892.326030 INFO <setup_settings>: Setting PID Settings
1484097895.328742 INFO <setup_test>: Setting Agitation to OFF and waiting 30 seconds
1484097895.329746 INFO <off>: Setting Agitation to Off


In [ ]:
r=run_the80Lalreadyworks()

In [ ]:
def test_mantoman(ip='71.189.82.196:85'):
    p = 0.1
    i = 0.01
    d = 0
    app = hello3.open_hello(ip)
    tests = []
    t1 = ManToManTest(p, i, d, 20, app, 15)
    t2 = ManToAutoTest(p,i,d,20, app, 15)
    t3 = OffToAutoTest(p,i,d,20, app)
    t4 = AutoToAutoTest(p,i,d,20, app, 15)
    tests = [t1, t2, t3, t4]
    for t in tests:
        t.mintime = 60
    r = PIDRunner(tests, wb_name="test.xlsx")
    return runagpid(r)
test_mantoman()

In [ ]:
app = hello3.open_hello('71.189.82.196:85')
t = ManToManTest(0.1, 0.01, 0, 10, app, 10)
r = PIDRunner([t], 'test.xlsx')
d = t.data
from time import time, sleep
start = time()
end = start + 50
def data():
    ag = app.gpmv()['agitation']
    pv = ag['pv']; op = ag['output']
    return time(), pv, op

while time() < end:
    p = data()
    d.append(p)
    sleep(0.5)
r.plotall()
r.chartbypid()